In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
import pandas as pd
import time

# Funktion zum Initialisieren des Edge-Browsers
def setup_driver():
    driver = webdriver.Edge(executable_path=r"") #Driver je nach Browser herunterladen (in diesem Fall Edge) und Pfad zu diesem angeben
    driver.get("https://omeka.unibe.ch/omeka-new/admin/user/add")
    return driver

# Funktion zum Einloggen
def login_to_omeka(driver, username, password):
    # Warte, bis die Login-Seite geladen ist, und finde die E-Mail- und Passwort-Felder
    email_field = WebDriverWait(driver, 5).until(
        EC.visibility_of_element_located((By.ID, "email"))
    )
    password_field = driver.find_element(By.ID, "password")
    
    # E-Mail und Passwort eingeben
    email_field.send_keys(username)
    password_field.send_keys(password)
    
    # Login-Button klicken
    password_field.send_keys(Keys.RETURN)
    
    # Warte, bis die Seite nach dem Login geladen ist
    WebDriverWait(driver, 5).until(
        EC.url_contains("/admin")  # Warte auf eine URL-Änderung, die das Admin-Dashboard anzeigt
    )

def add_user_to_omeka(driver, user_data):
    try:
        # Warten bis das E-Mail-Feld auf der Benutzerseite sichtbar ist
        email_field2 = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.ID, "email"))
        )
        display_name_field = driver.find_element(By.ID, "name")
        role_dropdown = driver.find_element(By.ID, "role")
        
        # Warten bis die Checkbox "Ist aktiv" verfügbar ist
        active_checkbox = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "is-active"))
        )

        # Werte aus der Excel-Datei einfügen
        email_field2.clear()  # Sicherstellen, dass das Feld leer ist
        email_field2.send_keys(user_data["Emailadresse für Omeka-Zugriff"])
        
        display_name_field.clear()
        display_name_field.send_keys(f"{user_data['Vorname']} {user_data['Name']}")
        
        role_dropdown.send_keys("Bearbeiter")  # Rollenfeld ausfüllen
        
        # Checkbox "Ist aktiv" anklicken, falls sie nicht aktiv ist
        if not active_checkbox.is_selected():
            active_checkbox.click()

        # "Hinzufügen"-Button lokalisieren über den Text
        add_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@id='page-actions']//button[text()='Hinzufügen']"))
        )

        # Scrollen zum Button, falls erforderlich
        driver.execute_script("arguments[0].scrollIntoView(true);", add_button)
        time.sleep(1)  # Kurz warten, um das Scrollen zu beenden

        # Button klicken
        add_button.click()
        print("Hinzufügen-Button geklickt, jetzt warten auf die Seite für den nächsten Benutzer...")

        # Warte eine kurze Zeit, damit der Benutzer gespeichert wird
        time.sleep(5)

        # Zur Seite zurückkehren, um den nächsten Benutzer hinzuzufügen
        driver.get("https://omeka.unibe.ch/omeka-new/admin/user/add")  # Zur Seite zurück navigieren

        # Warten, bis die Seite geladen ist
        WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.ID, "email"))
        )
        print("Zur Seite zurückgekehrt, fortfahren mit dem nächsten Benutzer.")

    except Exception as e:
        # Fehler beim Hinzufügen des Benutzers
        print(f"Fehler beim Hinzufügen eines Benutzers: {e}")
        driver.save_screenshot("error_before_add_user.png")  # Screenshot vor Fehler


def main():
    # Daten aus der Excel-Datei laden
    excel_path = r"" #Pfad zum lokal gespeicherten Excel
    sheet_data = pd.read_excel(excel_path).to_dict(orient="records")

    # Benutzername und Passwort für das Login
    username = "" #Benutzernamen einfügen!
    password = "" #Passwort einfügen!

    # Browser initialisieren
    driver = setup_driver()

    # Login durchführen
    login_to_omeka(driver, username, password)

    # Einträge nacheinander einfügen
    for user_data in sheet_data:
        add_user_to_omeka(driver, user_data)
        time.sleep(5)  # Kurze Pause zwischen den Einträgen, falls notwendig
    
    # Browser schließen
    driver.quit()

if __name__ == "__main__":
    main()